In [111]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

Import data

In [112]:
data = pd.read_csv('trip_NTS.csv' , header = 0)

C:\Users\bosst\AppData\Local\Temp\ipykernel_28416\515194137.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('trip_NTS.csv' , header = 0)


Create analysis dataframe for analysis

In [127]:
columnNeeded = ['TripType','DestRegion','Distance','Duration','AgeGroup','IncomeGroup','Reason','TotalSpendCan','ActVisitF', 'ActShopping', 'ActSightseeing', 'ActMuseum', 'ActHisSite','ActZoo', 'ActSport', 'ActFestival', 'ActPerformance', 'ActCasino', 'ActThemePark', 'ActMovie', 'ActDineout', 'ActNationalPark',  'ActBusiness','ActAllTerrainVehical', 'ActCanoeing', 'ActHiking', 'ActFishing', 'ActBeach', 'ActGolfing','ActCycling', 'ActSnowmobiling', 'ActSkiingDownhill','ActSkiingCrossCountry', 'ActPlaySport', 'ActOther', 'ActNone']
analysis_df = data[columnNeeded].dropna(axis = 0)

Transform datatype

In [115]:
transform_TripType = {
    "International sameday trip":'01',
    "International overnight trip":'02',
    "Domestic same day interprovincial trip":'03',
    "Domestic overnight interprovincial trip":'04',
    "Domestic same day intraprovincial trip":'05',
    "Domestic overnight intraprovincial trip":'06'
}
analysis_df['TripType'] = analysis_df['TripType'].map(transform_TripType)

In [116]:
transform_DestRegion = data[['DestRegion','TotalSpendCan']].groupby(by = 'DestRegion').mean().to_dict()['TotalSpendCan']
analysis_df['DestRegion'] = analysis_df['DestRegion'].map(transform_DestRegion)

In [117]:
transform_Reason = {
    '01': 'Holiday, leisure or recreation',
    '02': 'Visit friends or relatives',
    '03': 'Non-business conference, convention or trade show',
    '04': 'Shopping, non-routine',
    '05': 'To go to school or study, non-routine',
    '06': 'Medical, non-routine',
    '07': 'Religious reasons, non-routine',
    '08': 'Other personal, non-routine',
    '09': 'Business meeting, conference, convention, trade show',
    '10': 'Other business, non-routine'
}
analysis_df['Reason'] = analysis_df['Reason'].map({val: key for (key, val) in transform_Reason.items()})

In [118]:
transform_AgeGroup = {
    '01': '18 - 24 years old',
    '02': '25 - 34 years old',
    '03': '35 - 44 years old',
    '04': '45 - 54 years old',
    '05': '55 - 64 years old'
}
analysis_df['AgeGroup'] = analysis_df['AgeGroup'].map({val: key for (key, val) in transform_AgeGroup.items()})

In [119]:
transform_IncomeGroup = {
    '1': 'Less than $50,000',
    '2': '$50,000 to less than $70,000',
    '3': '$70,000 to less than $100,000',
    '4': '$100,000 and over'
}
analysis_df['IncomeGroup'] = analysis_df['IncomeGroup'].map({val: key for (key, val) in transform_IncomeGroup.items()})

Reset index

In [120]:
analysis_df = analysis_df.reset_index().drop('index',axis = 1)

Clean outliers

In [121]:
analysis_df = analysis_df[analysis_df['Duration'] <= (analysis_df['Duration'].mean() + 3*analysis_df['Duration'].std())].reset_index().drop('index',axis = 1)

In [122]:
analysis_df = analysis_df[analysis_df['Distance'] <= (analysis_df['Distance'].mean() + 3*analysis_df['Distance'].std())].reset_index().drop('index',axis = 1)

In [123]:
analysis_df = analysis_df[analysis_df['TotalSpendCan'] <= (analysis_df['TotalSpendCan'].mean() + 3*analysis_df['TotalSpendCan'].std())].reset_index().drop('index',axis = 1)

Stat Modeling

In [125]:
X = analysis_df[['Duration','IncomeGroup','Distance','DestRegion','Reason','ActVisitF', 'ActShopping', 'ActSightseeing', 'ActMuseum', 'ActHisSite','ActZoo', 'ActSport', 'ActFestival', 'ActPerformance', 'ActCasino','ActThemePark', 'ActDineout', 'ActNationalPark',  'ActBusiness','ActAllTerrainVehical', 'ActCanoeing', 'ActHiking', 'ActFishing', 'ActBeach',  'ActGolfing','ActCycling', 'ActSnowmobiling', 'ActSkiingDownhill','ActSkiingCrossCountry', 'ActPlaySport', 'ActOther', 'ActNone']]
y = analysis_df['TotalSpendCan']

In [126]:
X = sm.add_constant(X) # adding a constant
lin_reg = sm.OLS(y,X.astype(float))
model = lin_reg.fit()
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:          TotalSpendCan   R-squared:                       0.487
Model:                            OLS   Adj. R-squared:                  0.487
Method:                 Least Squares   F-statistic:                     4829.
Date:                Thu, 12 Oct 2023   Prob (F-statistic):               0.00
Time:                        23:32:11   Log-Likelihood:            -1.1542e+06
No. Observations:              162918   AIC:                         2.309e+06
Df Residuals:                  162885   BIC:                         2.309e+06
Df Model:                          32                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                  -107.05